In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
pic=Image(filename)

In [ ]:
pic

In [ ]:
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import tensorflow as tf
#Convolutional Neural Network (cifar10 )

# Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization,Activation,MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils import np_utils

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [ ]:


print('Training set shape:', X_train.shape)
print('Test set shape:', X_test.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train=X_train.reshape(-1,28,28,1)
X_test=X_test.reshape(-1,28,28,1)
num_classes=10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)
X_test.shape
print(y_train[0])
model=Sequential()

In [ ]:
X_test.shape

In [ ]:
#1
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (28, 28, 1)))
model.add(BatchNormalization())
#2
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#3
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#4
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))
#5
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))
#flattern layer and dense layer
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
from keras.preprocessing.image  import ImageDataGenerator
train_data=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_data=ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator(featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            rotation_range=15,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=False)

datagen.fit(X_train)

In [ ]:
#model compile and fit 
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0003, decay=1e-4),metrics=['accuracy'])

In [ ]:
                           
hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),
                           #steps_per_epoch=200,
                           epochs=10, #Increase this when not on Kaggle kernel
                           verbose=2,  #1 for ETA, 0 for silent
                           validation_data=(X_valid, y_valid)) #For speed
                           #callbacks=[annealer])
final_loss, final_acc = model.evaluate(X_valid, y_valid, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
#save your model 
import pickle
pickle.dump(model,open('file.pk1','wb'))   #here object  is model, write binary first write the 
file=pickle.load(open('file.pk1','rb'))    #load model with read mode  and initialize with var 

In [ ]:
file.predict('')   #now model is load and ready to predict 


In [ ]:
file.predict('')

In [ ]:
#take pictures from camera 
#convert rgb into gray
file = cv2.imread('/content/photo.jpg')
file = cv2.resize(file, (28, 28))
file = cv2.cvtColor(file, cv2.COLOR_BGR2GRAY)
file = file.reshape((-1, 28, 28,1))

In [ ]:
#plot pic
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(file.reshape(28,28))

In [ ]:
#predict that picture 
rslt=model.predict(file)


In [ ]:

if rslt[0][0]==1.0:
  print("1")
elif rslt[0][1]==1.0:
  print("2")
elif rslt[0][2]==1.0:
  print("3")
elif rslt[0][3]==1.0:
  print("4")
elif rslt[0][4]==1.0:
  print("5")
elif rslt[0][5]==1.0:
  print("6")
elif rslt[0][6]==1.0:
  print("7")
elif rslt[0][7]==1.0:
  print("8")
elif rslt[0][8]==1.0:
  print("9")
else:
  print("unknown class")

In [ ]:
#2 option  
import numpy as np
from keras.preprocessing import image
img_pred=image.load_img('/content/photo.jpg',target_size=(28,28))
img_pred=image.img_to_array(img_pred)
img_pred=np.expand_dims(img_pred,axis=0)
rslt=model.predict(img_pred)

In [ ]:
X_train=train_data.flow_from_directory('/content/drive/My Drive/cat_dog_cnn/PetImages',target_size=(64,64),batch_size=128)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#1
model1=Sequential()
model1.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (64, 64, 3)))
model1.add(BatchNormalization())
#2
model1.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model1.add(BatchNormalization())
#3
model1.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model1.add(BatchNormalization())
#4
model1.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPool2D(strides=(2,2)))
model1.add(Dropout(0.25))
#5
model1.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPool2D(strides=(2,2)))
model1.add(Dropout(0.25))
#flattern layer and dense layer
model1.add(Flatten())
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(5, activation='softmax'))

In [ ]:
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0003, decay=1e-4),metrics=['accuracy'])

In [ ]:
test_data=test_data.flow_from_directory('/content/drive/My Drive/cat_dog_cnn/PetImages/test_image',target_size=(64,64),batch_size=128)

In [ ]:
model1.fit_generator(X_train,epochs=10,validation_data=test_data)

In [ ]:
#convert image in to array
import cv2
def read_img(img_path):
    global img
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (64,64))
    return img

In [ ]:
read_img('/content/photo.jpg')
img.shape


In [ ]:
file = cv2.imread('/content/photo.jpg')
file = cv2.resize(file, (64,64))
#file = cv2.cvtColor(file, cv2.COLOR_BGR2GRAY)
file = file.reshape((-1, 64, 64,3))

In [ ]:
#import numpy as np
#from keras.preprocessing import image
#img_pred=image.load_img('/content/drive/My Drive/cat_dog_cnn/PetImages/Dog/1.jpg',target_size=(64,64))
#img_pred=image.img_to_array(img)
#img_pred=np.expand_dims(img_pred,axis=0)
#rslt=model1.predict(img_pred)

In [ ]:
rslt

In [ ]:
model1.predict_classes(img)

In [ ]:
model1.predict_classes(file)

In [ ]:
plt.imshow(file)

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

img = Image.open('photo.jpg').convert('LA')
img.save('greyscale.png')

plt.imshow(img,cmap='gray', vmin=0, vmax=255)

In [ ]:
model_json = model.to_json()

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

In [ ]:
loaded_model.predict(file)

In [ ]:
from keras.models import load_model 
model.save("cnn1.h5") 
#loaded_model = load_model("cnn1.h5") 

In [ ]:
!pip install flask-ngrok


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
app.run()